# Juan Manuel Deutsch, Cesar Felipe Giraldo, Julian Felipe Pulido

# Exercise 1

Para este ejercicio tenemos $X$ siendo la matriz base y $Σ$ como la matriz de covarianza de $X$

Tambien tenemos los componentes principales $u_1, u_2,...,u_n$, que son los autovectores o vectores propios de $Σ$ que tienen varianzas asociadas $u_i^TΣu_i$, donde $u_i^T$ es la transpuesta del vector propio $u_i$

Ahora tendremos una tranformación lineal de los datos $X$, siendo $AX$ donde $A$ es una transformación $m x m$

Ahora con la matriz transformada tenemos la matriz de covarianza transformada $AΣA^T$ cuyos componentes principales estan dados por $v_1, v_2,...,v_n$ estos siendo los autovectores cuyas varianza asociadas estan dadas por $v_i^TAΣA^Tv_i$

Teniendo en cuenta que necesitamos demostrar los componentes principales y como cambian tenemos con relacion a $AX$ y $X$

$u_1$ siendo componente principal cuya varianza asociada es $u_i^TΣu_i$ y siendo $v_1$ el componente principal cuya varianza asociada es $v_i^TAΣA^Tv_i$

Teniendo en cuenta que $u_1$ y $v_1$ estan relacionados por una transformación lineal tenemos que $v_1 = Tu_1$ para una matriz T

Ahora, se puede escribir la varianza asociada de $v_1$ como:

$$v_1^TAΣA^Tv_1 = (Tu_1)^TAΣA^T(Tu_1) = u_1^TT^TAΣA^TTu_1$$

Dado que $T$ es una matriz de transformacion lineal, $T^TAΣA^TT$ es otra matriz de covarianza $Σ'$

Por lo tanto ahora tenemos:

$$v_1^TAΣA^Tv_1 = u_1^TΣ'u_1$$

Concluyendo con esta demostración, tenemos que la varianza asociada $v_1$ depende de la matriz de covarianza $Σ'$ que es una version transformada de la matriz de covarianza original $Σ$, por lo cual, $Σ'$ es diferente a $Σ$, entonces la varianza asociada a estos van a ser diferentes, por lo que un valor propio esta ligado conjunto de datos original y el otro valor propio esta ligado al conjunto transformado

## 1.2)

Para esta demostracion tenemos $D$, como una matriz $f x c$ donde $f$ son filas y $c$ son columnas, ademas de $D'$ que es la matriz transformada con una transformación afin a una o mas columnas

Iniciando calculamos la matriz de correlación $corr(D)$
$$corr(D) = 1/n (D - D_u)^T(D - D_u)$$
Donde $D_u$ es la media de cada columna de $D$
De acuerdo a eso se los componentes principales por medio de $corr(D)$

Ahora realizamos el mismo proceso con la tranformación afin, que seria del siguiente modo:

$$D' = A*D+b$$

Donde $A$ es la transformación lineal y $b$ es el desplazamiento

Ahora calculamos la correlacion 

$$corr(D') = 1/n (D' - D'_u)^T(D' - D'_u)$$

Donde ahora $D'_u$ es la media de la matriz transformada, donde las columnas de $D'$ se ven afectadas por la transformación afin

Pero debemos tener en cuenta que para la covarianza los efectos de una transformación afin se dan de la siguiente forma:

$$cov(D') = A^Tcov(D)A$$

No obstante, estamos tratando es con matrices de correlación, las cuales son versiones escaladas de las matrices de covariaza, por lo cual la matriz de covarianza tambien afecta a la matriz de correlación con la misma transformación afin

Por lo cual los autovectores o vectores propios de $corr(D')$ son los mismos que $corr(D)$


# Exercise 2

In [9]:
import numpy as np
D = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
red = 1
def PCA(D, red):

    mu = list(sum(row)/len(row) for row in zip(*D))
    n = D.shape[0]
    centeredData = D - mu
    cov = np.dot(centeredData.T, centeredData) / (n)
    eigenValues, eigenVectors = np.linalg.eig(cov)
    print(eigenVectors)
    if(red < 1):
        print('lo que sea')
    else: 
        r = red
    Ur = eigenVectors[:, :r]
    print(Ur)

PCA(D, red)


[[ 5.77350269e-01 -3.27465273e-18  8.09500438e-01]
 [ 5.77350269e-01 -7.07106781e-01 -4.97117858e-01]
 [ 5.77350269e-01  7.07106781e-01 -3.12382580e-01]]
[[0.57735027]
 [0.57735027]
 [0.57735027]]


In [2]:
#PUNTO JULIAn
import numpy as np

def PCA(D, red):

    mu = np.mean(D, axis=0)

    Z = D - mu
    
    cov_matrix = np.cov(Z, rowvar=False, bias=False)
    
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    
    if red < 1:

        total_variance = np.sum(eigenvalues)
        variance_top = red * total_variance
        
        cumulative_variance = 0
        r = 0
        for i in range(len(eigenvalues)):
            cumulative_variance += eigenvalues[i]
            if cumulative_variance >= variance_top:
                r = i + 1
                break
    else:

        r = red
    
    print(r)
  
    top_eigenvectors = eigenvectors[:, :r]

 
    print(top_eigenvectors)
    
    A = np.dot(Z, top_eigenvectors)
    
    v = np.sum(eigenvalues[:r])
    
    return A, v

#Prueba

D = np.array([[1, 2, 3],
               [4, 5, 6],
               [7, 8, 9],
               [10, 11, 12],
               [13, 14, 15]])

# Aplicamos PCA para reducir a 2 dimensiones conservando el 50% de la varianza
A, variance = PCA(D, 0.5)
print(A)
print(variance)

1
[[-0.57735027]
 [-0.57735027]
 [-0.57735027]]
[[ 10.39230485]
 [  5.19615242]
 [  0.        ]
 [ -5.19615242]
 [-10.39230485]]
67.5


### PREPARACION DE LA DATA


In [3]:
import numpy as np
import pandas as pd
adf = pd.read_csv("amazon.csv")
bndf = pd.read_csv("baseball_numeric.csv")
fdf = pd.read_csv("fabert.csv")
idf = pd.read_csv("iris.csv")

adf2 = adf.drop('Class', axis=1)
fdf2 = fdf.drop('class', axis=1)
idf2 = idf.drop('species', axis=1)

#Quitamos las columnas que no tienen valores numericos

NameError: name 'pd' is not defined